In [233]:
import pandas as pd
from dotenv import load_dotenv
from utils import get_engine
from sqlalchemy import text
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [234]:
load_dotenv()
engine = get_engine(echo_arg=True)

In [235]:
with engine.begin() as conn:
    query = text("""SELECT * FROM "CO2_bulding_filtered_feauturs_selection"; """)
    df = pd.read_sql(query, conn)

2023-06-15 20:27:49,997 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-06-15 20:27:50,000 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-15 20:27:50,043 INFO sqlalchemy.engine.Engine select current_schema()
2023-06-15 20:27:50,045 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-15 20:27:50,090 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-06-15 20:27:50,092 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-15 20:27:50,134 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-15 20:27:50,145 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [236]:
X_cols = ['YearBuilt', 'Have_Stream_Energy', 'Have_Electricity_Energy',
       'Have_NaturalGas_Energy', 'BuildingType', 'PrimaryPropertyType',
       'Neighborhood', 'NumberofBuildings',
       'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking',
       'PropertyGFABuilding_s_', 'LargestPropertyUseTypeGFA']
X = df[X_cols]

In [237]:
y = df['TotalGHGEmissions']

In [238]:
bool_cols = ['Have_Stream_Energy', 'Have_Electricity_Energy','Have_NaturalGas_Energy']
X[bool_cols]= X[bool_cols].apply(lambda x: x.apply(lambda x: 1 if x else 0))

/tmp/ipykernel_255155/2630812360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[bool_cols]= X[bool_cols].apply(lambda x: x.apply(lambda x: 1 if x else 0))


In [239]:
X_cat = X.select_dtypes(include=[object])
X_num = X.select_dtypes(exclude=[object])

In [240]:
preparation = ColumnTransformer(transformers=[
    ('tf_cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), X_cat.columns),
    ('tf_num', RobustScaler(), X_num.columns)
])

In [241]:
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

In [243]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_tf_train = preparation.fit_transform(X_train)
X_tf_test = preparation.fit_transform(X_test)

In [244]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_tf_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_71 (Dense)            (None, 128)               7808      
                                                                 
 dense_72 (Dense)            (None, 256)               33024     
                                                                 
 dense_73 (Dense)            (None, 256)               65792     
                                                                 
 dense_74 (Dense)            (None, 256)               65792     
                                                                 
 dense_75 (Dense)            (None, 1)                 257       
                                                                 
Total params: 172,673
Trainable params: 172,673
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
# checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
# callbacks_list = [checkpoint]

In [246]:
NN_model.fit(X_tf_train, y, epochs=100, batch_size=32, validation_split = 0.2)

Epoch 1/100
68/68 [==============================] - 4s 21ms/step - loss: 136.2131 - mean_absolute_error: 136.2131 - val_loss: 55.0093 - val_mean_absolute_error: 55.0093
Epoch 2/100
68/68 [==============================] - 1s 13ms/step - loss: 124.2777 - mean_absolute_error: 124.2777 - val_loss: 54.2365 - val_mean_absolute_error: 54.2365
Epoch 3/100
68/68 [==============================] - 1s 10ms/step - loss: 122.8308 - mean_absolute_error: 122.8308 - val_loss: 53.9626 - val_mean_absolute_error: 53.9626
Epoch 4/100
68/68 [==============================] - 1s 10ms/step - loss: 122.4714 - mean_absolute_error: 122.4714 - val_loss: 54.2809 - val_mean_absolute_error: 54.2809
Epoch 5/100
68/68 [==============================] - 1s 10ms/step - loss: 122.4450 - mean_absolute_error: 122.4450 - val_loss: 54.6223 - val_mean_absolute_error: 54.6223
Epoch 6/100
68/68 [==============================] - 1s 11ms/step - loss: 122.5143 - mean_absolute_error: 122.5143 - val_loss: 54.0169 - val_mean_abso

In [247]:
y_pred = NN_model.predict(X_tf_test)

22/22 [==============================] - 0s 4ms/step


In [248]:
from sklearn.metrics import r2_score
print(r2_score(y_test, y_pred))

-0.17956495164765673
